In [1]:
import ROOT
ROOT.RooMsgService.instance().setGlobalKillBelow(5)
%jsroot on

from matplotlib import pyplot
%matplotlib inline

Welcome to JupyROOT 6.07/07


## Higgs SR

In [2]:
infile = ROOT.TFile.Open("output/SRselection.root")
w = infile.Get("w")
mc = w.obj("ModelConfig")
data = w.data("data")
mc.LoadSnapshot()
x = w.var("x")

## Profile Likelihood on number of signal events

In [5]:
pl = ROOT.RooStats.ProfileLikelihoodCalculator(data,mc)
pl.SetConfidenceLevel(0.683)

In [13]:
mc.LoadSnapshot()
interval = pl.GetInterval()

In [14]:
firstPOI = mc.GetParametersOfInterest().first()
lowerLimit = interval.LowerLimit(firstPOI)
upperLimit = interval.UpperLimit(firstPOI)

In [8]:
print "68% interval on {0} is [{1},{2}]".format(firstPOI.GetName(),lowerLimit,upperLimit)

68% interval on n_sig is [300.405555929,589.564440644]


In [18]:
plot = ROOT.RooStats.LikelihoodIntervalPlot(interval)
plot.SetRange(200,700)
plot.SetNPoints(50)
c = ROOT.TCanvas()
ROOT.gPad.SetLogy(True)
plot.Draw()
c.Draw()

Error in <THistPainter::PaintInit>: log scale is requested but maximum is less or equal 0 (-1.000000)


### Bayesian Markov Chain Monte Carlo Calculator 

In [20]:
nsig = w.var("n_sig")
nsig.setRange(300,600)

Now we define our MCMC calculator. Again looking at 68.3% confidence level.

In [21]:
mcmc = ROOT.RooStats.MCMCCalculator(data,mc)
mcmc.SetConfidenceLevel(0.683)

Now MCMC methods have various implementations and approaches. Here we must propose a function. The sequential proposal given is usually fairly robust.

In [22]:
sp = ROOT.RooStats.SequentialProposal(0.1)
mcmc.SetProposalFunction(sp)

If you're interested in this I'd recommend looking it up. Here I just define lots of parameters. SetNumIters is related to the number of iterations of the Metropolis-Hastings algorithm SetBurnInSteps concerns how many interations are needed to avoid local minima due to low stats We can construct either a one-sided Bayesian interval (SetLetSideTailFraction=0) or central bayesian interval (SetLetSideTailFraction=0.5)

In [23]:
mcmc.SetNumIters(100000)
mcmc.SetNumBurnInSteps(1000)
mcmc.SetLeftSideTailFraction(0.5)

Now we run the calculator! (may take a while depending on the number of steps)

In [24]:
interval = mcmc.GetInterval()

Metropolis-Hastings progress: ....................................................................................................


Now we plot and print!

In [25]:
firstPOI = mc.GetParametersOfInterest().first()

print "68% interval on {0} is : [{1},{2}]".format(firstPOI.GetName(),interval.LowerLimit(firstPOI),interval.UpperLimit(firstPOI))

68% interval on n_sig is : [346.779036895,527.144738904]


In [26]:
c = ROOT.TCanvas()
plot = ROOT.RooStats.MCMCIntervalPlot(interval)
plot.Draw()
c.Draw()

